In [50]:
import ee
import geopandas as gpd
import eeconvert

In [65]:
VECTOR = r'../vector/Sampling_sites.shp'
OUTFILE = r'../Sampling_sites.shp'
CRS = 'EPSG:4326'

In [52]:
ee.Initialize()

#### Load dataset 

In [53]:
df = gpd.read_file(VECTOR)

In [54]:
df

,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,geometry
0,DB-IN,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.50874 68.51269 28.17294)
1,DB-EX,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.50844 68.51143 16.44608)
2,DB-NON,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.49628 68.51216 53.53075)
3,UP-IN,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.51461 68.51278 0.00000)
4,UP-NON,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.48839 68.50447 0.00000)


#### Convert geodataframe to feature collection 

In [55]:
# eeconvert only accepts 2D Points
df['X'] = df.geometry.x
df['Y'] = df.geometry.y
df['geometry'] = gpd.points_from_xy(df['X'], df['Y'], crs=CRS)

In [56]:
fc = eeconvert.gdfToFc(df)

#### access GEE Layer

In [57]:
arcticDEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")

In [58]:
output = arcticDEM.select("elevation").sampleRegions(collection=fc)

In [59]:
agg = output.aggregate_array("elevation")

In [60]:
elevation = agg.getInfo()

#### write output 

In [62]:
df['X'] = df.to_crs(crs=CRS).geometry.x
df['Y'] = df.to_crs(crs=CRS).geometry.y
df['Z'] = elevation

In [63]:
df['geometry'] = gpd.points_from_xy(df['X'], df['Y'], z=df['Z'], crs=CRS)

In [64]:
df

,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,geometry,X,Y,Z
0,DB-IN,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.50874 68.51269 7.32951),161.508737,68.512694,7.329514
1,DB-EX,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.50844 68.51143 8.49137),161.508437,68.511427,8.491371
2,DB-NON,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.49628 68.51216 9.15307),161.496279,68.512158,9.153074
3,UP-IN,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.51461 68.51278 10.14383),161.514611,68.512778,10.143826
4,UP-NON,None,None,None,None,None,-1,0,-1,None,None,POINT Z (161.48839 68.50447 18.19840),161.488391,68.504469,18.198404


In [66]:
df.to_file(OUTFILE, index=False)